In [1]:
from q_lab_toolbox.target_systems import DecaySystem
from q_lab_toolbox.hamiltonians import create_hamiltonian
from q_lab_toolbox.jump_operators import create_jump_operators

In [2]:
decay_system = DecaySystem(ryd_interaction=0.1, omegas=(0.2, 0.4), m=2, gammas=(0.3, 0.35))

In [3]:
H = create_hamiltonian(decay_system)
An = create_jump_operators(decay_system)

In [4]:
from q_lab_toolbox.initial_states import _rho_rand_haar

rho0 = _rho_rand_haar(m=2, seed=3)

In [5]:
rho0

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.43099531+0.j         -0.03645119+0.10609661j -0.20985646+0.32143067j
  -0.27240032+0.10532724j]
 [-0.03645119-0.10609661j  0.02920027+0.j          0.09687397+0.02447481j
   0.04896615+0.05814784j]
 [-0.20985646-0.32143067j  0.09687397-0.02447481j  0.34190027+0.j
   0.21118646+0.15186759j]
 [-0.27240032-0.10532724j  0.04896615-0.05814784j  0.21118646-0.15186759j
   0.19790415+0.j        ]]

In [6]:
N=10

In [1]:
from q_lab_toolbox.target_systems import DecaySystem
from q_lab_toolbox.hamiltonians import create_hamiltonian
from q_lab_toolbox.jump_operators import create_jump_operators
from q_lab_toolbox.initial_states import _rho_rand_haar
import numpy as np
from q_lab_toolbox.training_data import mk_training_data_states
from q_lab_toolbox.readout_operators import _order_n_observables
from q_lab_toolbox.channels import GateBasedChannel
from q_lab_toolbox.unitary_circuits import HardwareAnsatz

In [2]:
decay_system = DecaySystem(
    ryd_interaction=0.1, omegas=(0.2, 0.4), m=2, gammas=(0.3, 0.35)
)

H = create_hamiltonian(decay_system)
An = create_jump_operators(decay_system)


rho00 = _rho_rand_haar(m=2, seed=3)
rho10 = _rho_rand_haar(m=2, seed=4)

N = 3
ts = np.linspace(0, 10, N + 1)


rhoss = mk_training_data_states([rho00, rho10], ts, decay_system)


circuit = HardwareAnsatz(
    m=2, n_qubits=5, depth=5, gate_type="ryd", structure="triangle"
)

Os = _order_n_observables(m=2, n=1)

training_data = rhoss, Os

print(circuit.J_from_states(circuit.init_flat_theta(), training_data))

O: Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
trace: 0.0
trace: -7.216449660063518e-15
trace: -1.3961054534661343e-14
trace: -2.09277040141842e-14
O: Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
trace: 0.0
trace: (0.473873732375372-1.1102230246251565e-16j)
trace: (0.37040069885809856-6.938893903907228e-17j)
trace: (0.27945682528672283-5.551115123125783e-17j)
O: Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]]
trace: 0.0
trace: 0.16151260159112235
trace: 0.07292321656233185
trace: (-0.13520552335821345-4.163336342344337e-17j)
O: Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper

c:\Users\Admin\Desktop\BFP\quantum-channel-approximation\q_lab_toolbox\training_data.py:84: ComplexWarning: Casting complex values to real discards the imaginary part
  Ess[k, :] = np.array([(O * qt.Qobj(rho, dims=dims)).tr() for rho in rhos])
